In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle 

In [3]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Preprocess the data
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)

In [5]:
# Encode categorical variables 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
# Combine one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [17]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
# Save the encoders and scaler for later use 
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [20]:
import pickle

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


## ANN Regression Problem Statement 


In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Build the model
model = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Output layer for regression  # When we don't apply any activation function the default activation function that is applied is linear activation function. Linear activation function is for regression, which uses the output that we get from the model
])

## Compile the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics =['mae'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [27]:
# Set ip Early Stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)


In [29]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 100383.7578 - mae: 100383.7578 - val_loss: 98538.6406 - val_mae: 98538.6406
Epoch 2/100
250/250 [==============================] - 0s 873us/step - loss: 99700.6484 - mae: 99700.6484 - val_loss: 97149.5938 - val_mae: 97149.5938
Epoch 3/100
250/250 [==============================] - 0s 833us/step - loss: 97274.0391 - mae: 97274.0391 - val_loss: 93573.7422 - val_mae: 93573.7422
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 92463.3828 - mae: 92463.3828 - val_loss: 87547.6875 - val_mae: 87547.6875
Epoch 5/100
250/250 [==============================] - 0s 925us/step - loss: 85356.6953 - mae: 85356.6953 - val_loss: 79588.0234 - val_mae: 79588.0234
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 76799.7578 - mae: 76799.7578 - val_loss: 70989.6641 - val_mae: 70989.6641
Epoch 7/100
250/250 [==============================] - 0s 999us/step - loss: 68153.0938 - mae: 681

In [30]:
%load_ext tensorboard

In [33]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 63335), started 0:00:19 ago. (Use '!kill 63335' to kill it.)

In [34]:
## Evaluate the model on test data
testloss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 491us/step - loss: 50321.9297 - mae: 50321.9297
Test MAE : 50321.9296875


In [35]:
model.save('regression_model.h5')

/Users/vibhupratap/Documents/Projects/Churn_Modelling_Project-ANN-/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
